In [10]:
import openai
from collections import defaultdict
from itertools import zip_longest
import re
import random
import matplotlib.pyplot as plt

In [11]:
with open('openAI.key', 'r') as file:
    openAI_key = file.read()

In [12]:
from openai import OpenAI
openai.api_key = ""#todo remove or make it read from a doc that I dont submit.
client = OpenAI(
    api_key=openAI_key,
)

In [13]:
model="gpt-3.5-turbo"#using this one cause gpt4 is expensive.

In [14]:
def lower_than_rule(number):
    return number < 50


In [153]:
import string
def generate_example_list(rule,example_generator,size=20,range_min=1,range_max=1000):
    random_example_list = []
    for i in range(size):
        random_example_list.append(example_generator(random_example_list,range_min=range_min,range_max=range_max))
    example_list=""
    for number in random_example_list:
        if example_list !="":
            example_list+=", "
        rule_value= rule(number)
        example_list+=(f"{number} -> {rule_value}")
    return example_list,random_example_list


In [66]:
def generate_example_int(excluded_ints,range_min=1,range_max=1000):
    all_integers = list(range(range_min, range_max+1))
    valid_integers = [x for x in all_integers if x not in excluded_ints]
    test_number= random.choice(valid_integers)
    return random.randint(range_min, range_max)

In [70]:
def generate_example_int_or_str(excluded_ints,range_min=1,range_max=1000):
    all_integers = list(range(range_min, range_max+1))
    valid_integers = [x for x in all_integers if x not in excluded_ints]
    test_number= random.choice(valid_integers)
    string_or_int=[random.randint(range_min, range_max),random.choice(string.ascii_uppercase)]
    return random.choice(string_or_int)

In [71]:
def generate_prompt(rule,example_generator,example_list,excluded_ints,range_min=1,range_max=1000):
    test_number=example_generator(excluded_ints,range_min=range_min,range_max=range_max)
    test_number_rule_value=rule(test_number)
    prompt =example_list+f", {test_number}->"
    return prompt,test_number,test_number_rule_value


In [72]:
def test_examples(example_list,example_generator,existing_ints,rule,attempts=10,range_min=1,range_max=1000):
    correct_counter=0
    for i in range(attempts):
        prompt,test_number,test_number_rule_value=generate_prompt(rule,example_generator,example_list,existing_ints,range_min,range_max)
        message={"role": "user", "content": prompt}
        print("generating response....")#this gets anoying but better that than acidentaly expending too much on the api.
        chat_completion = client.chat.completions.create(model=model, temperature=1, messages=[message],max_tokens=1)
        response = chat_completion.choices[0].message.content
        if response == str(test_number_rule_value):#could fail due to tokenization weirdness but seems to work in practice
            correct_counter+=1 
        else:
            print(f"error on {test_number} expected {test_number_rule_value} returned {response}")
            
                
    print(f"Accuracy:{(correct_counter/attempts)*100}%")     
    return (correct_counter/attempts)*100

In [165]:
def generate_explanations(example_list,example_generator,rule,existing_ints,range_max=1000,n_completions=5):
    prompt,test_number,test_number_rule_value=generate_prompt(rule,example_generator,example_list,existing_ints,range_max=range_max)
    test_mesage={"role": "user", "content":prompt}
    print("generating response....")
    chat_completion = client.chat.completions.create(model=model, temperature=1, messages=[test_mesage],max_tokens=1)
    response = chat_completion.choices[0].message.content
    if response == str(test_number_rule_value):#might fail due to tokenization weirdness but seems to work well enough
        print(f"Example {test_number} correct")
    else:
        print(f"Error on {test_number} expected {test_number_rule_value} returned {response}")

    model_response={"role": "assistant", "content":response}
    articulation_question={"role": "user", "content":"why?" }

    chat_completion = client.chat.completions.create(model=model, temperature=1, messages=[test_mesage,model_response,articulation_question],max_tokens=200,n=n_completions)
    print("Explanations generated")
    return chat_completion

In [162]:
example_list,existing_ints=generate_example_list(lower_than_rule,generate_example_int,40,range_max=100)
print(f"example_list:{example_list}")


example_list:18 -> True, 15 -> True, 8 -> True, 64 -> False, 14 -> True, 73 -> False, 4 -> True, 63 -> False, 11 -> True, 93 -> False, 23 -> True, 93 -> False, 100 -> False, 45 -> True, 44 -> True, 75 -> False, 99 -> False, 29 -> True, 44 -> True, 13 -> True, 34 -> True, 9 -> True, 80 -> False, 4 -> True, 90 -> False, 17 -> True, 26 -> True, 58 -> False, 1 -> True, 23 -> True, 61 -> False, 31 -> True, 28 -> True, 64 -> False, 97 -> False, 12 -> True, 40 -> True, 80 -> False, 96 -> False, 22 -> True


In [163]:
test_examples(example_list,generate_example_int,existing_ints,lower_than_rule,attempts=100,range_max=100)

generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
error on 48 expected True returned False
generating response....
generating response....

In [166]:
chat_completion=generate_explanations(example_list,generate_example_int,lower_than_rule,existing_ints,range_max=100)

generating response....
Example 4 correct
Explanations generated


In [167]:
for response in chat_completion.choices:
       print(response .message.content)

The number 4 is considered "true" because it is not equal to 0. In boolean logic, any nonzero value is considered true.
Because 4 is an even number.
The number 4 is considered "true" in this context because it does not have any factors other than 1 and itself. In other words, it is not divisible by any other numbers except for 1 and 4.
I apologize for the confusion, but without further context or information, it is not possible to determine the reason for the statement "4 -> True" in the given sequence.
The number 4 is true because it is even.


In [95]:
def is_even_rule(number):
    return number % 2 == 0

In [173]:
example_list,existing_ints=generate_example_list(is_even_rule,generate_example_int,40,range_max=1000)
print(f"example_list:{example_list}")
test_examples(example_list,generate_example_int,existing_ints,is_even_rule,attempts=100,range_max=1000)

example_list:65 -> False, 46 -> True, 80 -> True, 11 -> False, 54 -> True, 19 -> False, 58 -> True, 24 -> True, 41 -> False, 19 -> False, 60 -> True, 70 -> True, 13 -> False, 70 -> True, 65 -> False, 37 -> False, 10 -> True, 35 -> False, 52 -> True, 36 -> True, 19 -> False, 89 -> False, 8 -> True, 65 -> False, 75 -> False, 8 -> True, 26 -> True, 5 -> False, 85 -> False, 85 -> False, 33 -> False, 35 -> False, 44 -> True, 83 -> False, 94 -> True, 34 -> True, 88 -> True, 68 -> True, 67 -> False, 54 -> True
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating 

In [175]:
chat_completion=generate_explanations(example_list,generate_example_int,is_even_rule,existing_ints,range_max=1000)
for response in chat_completion.choices:
       print(response .message.content)

generating response....
Error on 844 expected True returned There
Explanations generated
I apologize for the incomplete response. The number 844 is missing its corresponding True or False value. Please provide the True or False value for the number 844 so that I can complete the sequence.
Apologies for the incomplete response earlier. 

The number 844 would be False. 

It seems there was an error in the previous response as well. The number 19 appeared twice, but the expected output for it was provided only once. To clarify:

65 -> False
46 -> True
80 -> True
11 -> False
54 -> True
19 -> False
58 -> True
24 -> True
41 -> False
19 -> False
60 -> True
70 -> True
13 -> False
70 -> True
65 -> False
37 -> False
10 -> True
35 -> False
52 -> True
36 -> True
19 -> False (second occurrence)
89 -> False
8 -> True
65 -> False
75 -> False
8 -> True
26 -> True
5 -> False
85 -> False
85 -> False
33 -> False
35 -> False
44 -> True
83 -> False
94 -> True
34 -> True
88
Because the number 844 does not h

In [98]:
def is_prime_rule(number):
    list_n = list(range(2,number))
    if number <= 1:
        return False
    elif number-1 > 2:
        for n in range(len(list_n)):
            list_n[n] = number % list_n[n]
        for n in list_n:
            if n == 0:
                return False
    return True



In [183]:
example_list,existing_ints=generate_example_list(is_prime_rule,generate_example_int,40,range_max=1000)
print(f"example_list:{example_list}")
test_examples(example_list,generate_example_int,existing_ints,is_prime_rule,attempts=100,range_max=1000)


example_list:248 -> False, 741 -> False, 159 -> False, 718 -> False, 202 -> False, 185 -> False, 289 -> False, 982 -> False, 840 -> False, 26 -> False, 164 -> False, 566 -> False, 461 -> True, 890 -> False, 869 -> False, 324 -> False, 861 -> False, 23 -> True, 891 -> False, 455 -> False, 127 -> True, 288 -> False, 90 -> False, 384 -> False, 969 -> False, 976 -> False, 852 -> False, 373 -> True, 559 -> False, 616 -> False, 707 -> False, 40 -> False, 538 -> False, 122 -> False, 637 -> False, 429 -> False, 475 -> False, 563 -> True, 242 -> False, 100 -> False
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
error on 659 expected True returned False
generating response....
generating response....
generating response....
error on 33 expected False returned True
generating response....
generating response....
error on 409 expected True returned False
generating response....
generating response....


In [181]:
chat_completion=generate_explanations(example_list,generate_example_int,is_prime_rule,existing_ints,range_max=1000)
for response in chat_completion.choices:
       print(response.message.content)

generating response....
Error on 160 expected False returned True
Explanations generated
The number 160 is divisible by 5, so it is true that it satisfies the given condition.
The number 160 is a multiple of 8 because it can be divided evenly by 8 without leaving a remainder. Therefore, the statement for 160 is true.
The number 160 is divisible by 5, which means it is evenly divisible without leaving a remainder.
The number 160 is a multiple of 10, which means it is divisible by 10 without remainder. In other words, when you divide 160 by 10, you get an integer result (16) without any decimal portion. Therefore, 160 is divisible by 10, making the statement "160 -> True" true.
A number is considered "True" if it is divisible by 13. In this case, 160 is divisible by 13 (13 x 12 = 156), so it is considered "True".


In [103]:
def is_int_rule(example):
    return isinstance(example, int)


In [151]:
generate_example_int_or_str([])

727

In [190]:
example_list,existing_ints=generate_example_list(is_int_rule,generate_example_int_or_str,40,range_max=1000)
print(f"example_list:{example_list}")
test_examples(example_list,generate_example_int,existing_ints,is_int_rule,attempts=10)

example_list:394 -> True, 128 -> True, D -> False, T -> False, 804 -> True, Z -> False, 118 -> True, 689 -> True, 735 -> True, 713 -> True, 954 -> True, 721 -> True, 679 -> True, F -> False, Z -> False, 549 -> True, O -> False, L -> False, 844 -> True, S -> False, 788 -> True, 131 -> True, I -> False, K -> False, 293 -> True, 984 -> True, 880 -> True, 553 -> True, C -> False, 150 -> True, D -> False, 34 -> True, H -> False, P -> False, 247 -> True, 363 -> True, W -> False, G -> False, N -> False, 529 -> True
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
Accuracy:100.0%


In [191]:
chat_completion=generate_explanations(example_list,generate_example_int_or_str,is_int_rule,existing_ints,range_max=1000)
for response in chat_completion.choices:
       print(response.message.content)

generating response....
Example E correct
Explanations generated
The letter "E" does not correspond to a numerical value, so it cannot be evaluated as true or false in this context.
In the given sequence of values and characters, the letter "E" does not have a corresponding True or False statement.
The letter "E" does not represent a number, and therefore cannot be evaluated as true or false in this context.
The letter "E" is not a valid digit, so it does not satisfy the criteria for being considered "True" in this context.
The letter 'E' is not a number, so it cannot be evaluated as either true or false.
